In [ ]:
import os 
import sys
import cv2
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.spatial.transform import Rotation as R

NOTEBOOKS_DIR = os.path.abspath("./")
sys.path.append(f"{NOTEBOOKS_DIR}/..")
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


sys.path.append(f"{NOTEBOOKS_DIR}/..")

import sphere_snap.utils as snap_utils
import sphere_snap.sphere_coor_projections as sphere_proj
from sphere_snap.snap_config import SnapConfig, ImageProjectionType
from sphere_snap.sphere_snap import SphereSnap
import sphere_snap.reprojections as rpr


In [ ]:
def show_img(img):
    plt.figure(figsize=(13, 13))
    plt.imshow(img)
    plt.show()

def show_imgs(imgs, size = 14, nb_cols=2, title_txt= None, fontsize=10, imgs_text = None):
    """
    Display a gird of images 
    :param imgs: the list of images to be displayed 
    :param size: display size of a cell
    :param nb_cols: number of columns
    """

    row_size = int(np.ceil(len(imgs)/nb_cols))
    fig = plt.figure(figsize=(size,size))
    axes_pad = 0.1 if imgs_text is None else 0.5

    if title_txt is not None:
        fig.suptitle(title_txt, fontsize=fontsize)
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(row_size, nb_cols),  # creates 2x2 grid of axes
                     axes_pad=axes_pad,  # pad between axes in inch.
                     )

    for idx, data in enumerate(zip(grid, imgs)):
        ax, img  = data
        if imgs_text is not None:
             ax.set_title(imgs_text[idx], fontdict={'fontsize': 15, 'fontweight': 'medium'}, loc='center', color = "k")
        ax.imshow(img)
    plt.show()
    
def rot(yaw, pitch):
    return R.from_euler("yxz",[yaw,-pitch,0], degrees=True).as_quat()

def blend_img(a, b, alpha=0.8):
    return (alpha*a + (1-alpha) * b).astype(np.uint8)

In [ ]:
equi_photo = plt.imread(f"{NOTEBOOKS_DIR}/../data/equirectangular.jpg")
fixed_size = (2000,1000)
equi_photo = cv2.resize(equi_photo, fixed_size)
half_equi_img = equi_photo[:,500:-500,:]
show_imgs([equi_photo], nb_cols=1)

### Snap to perspective from equirectangular

In [ ]:
snap_config = SnapConfig( rot(-10,-3), (480,853),(125,95), equi_photo.shape[:2], source_img_type=ImageProjectionType.EQUI)
snap_test = SphereSnap(snap_config)
persp_img = snap_test.snap_to_perspective(equi_photo)
show_img(persp_img)

In [ ]:
show_imgs([half_equi_img], nb_cols=1)

In [ ]:
snap_config = SnapConfig( rot(-10,-3), (480,853),(125,95), half_equi_img.shape[:2], source_img_type=ImageProjectionType.HALF_EQUI)
snap_test = SphereSnap(snap_config)
persp_img = snap_test.snap_to_perspective(half_equi_img)
show_img(persp_img)

In [ ]:
import cv2

point = (400,500)
half_equi_img = cv2.circle(half_equi_img, point, radius=0, color=(0, 0, 255), thickness=15)
show_img(half_equi_img)

In [ ]:
point_spherical = sphere_proj.halfequi_coor2spherical(np.array([point]), half_equi_img.shape[:2])
xyz_point = sphere_proj.spherical2xyz(point_spherical)[0]
xyz_point

In [ ]:

snap_config = SnapConfig.createFromPoint( xyz_point, (480,853),(125,95), half_equi_img.shape[:2], source_img_type=ImageProjectionType.HALF_EQUI)
snap_test = SphereSnap(snap_config)
persp_img = snap_test.snap_to_perspective(half_equi_img)
show_img(persp_img)

### Reproject equirectangular into 6 planar images of 90 degrees FoV (Cubemap)

In [ ]:
cube_faces_imgs = rpr.equi2cubemap(equi_photo)
show_imgs(cube_faces_imgs, nb_cols=6, size=20)

### Reproject a planar image into equirectangular

In [ ]:
reconstructed_equi = rpr.cubemap2hequi(cube_faces_imgs)
show_img(reconstructed_equi)


In [ ]:
tv_img = rpr.equi2tv(equi_blend, np.identity(3), (1000,1000), (90,90), 10, 2)
show_img(tv_img)